In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-small-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
assert torch.cuda.is_available(), "This model needs a GPU to run ..."
device = torch.cuda.current_device()
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
    "pad_token_id": tokenizer.eos_token_id
}

def promptModel(prompt, max_tokens=500, temperature=0.0, do_sample=False):
    messages = [
        {"role": "user", "content": prompt},
    ]
    generation_args = {
        "max_new_tokens": max_tokens,
        "return_full_text": False,
        "temperature": temperature,
        "do_sample": do_sample,
        "pad_token_id": tokenizer.eos_token_id
    }
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

In [ ]:
_prompt_   = 'Translate the following text into an ontology represented as JSON.  Do not include the schema context.  Do not explain your response.'
_sentence_ = 'The army of people Asterian (APA) is paramilitary organization which has been busy with terrorist activities ' + \
             'which are financed by its criminal ventures, which include drug trafficking.'
_response_ = promptModel(_prompt_ + '\n\n' + _sentence_, max_tokens=4096)
print(_response_)

In [ ]:
import json
json.loads(_response_[_response_.index('{'):_response_.rindex('}')+1])